# Weekly All-hands Meeting Report

## Import data

In [15]:
import pandas as pd
import numpy as np

# read file
df = pd.read_excel('Sales data template.xlsx')

# Need to be updated
Days_in_lastmonth =  31
Days_thismonth = 31
Days_analysed = 31
Thismonth = 'August'
Lastmonth = 'July'
Lastmonth_prorata = 'Pro rata July'
paidspent_thismonth = 44342

## Data Preparation

In [16]:
# Date
df['day'] = pd.to_datetime(df['day'])
df['month'] = df['day'].dt.strftime('%B')
# select the data inthis month
df = df[df['month'] == Thismonth]

# change the currency for uk

df['gross_sales_sgd'] = df.apply(lambda row: row['gross_sales'] if row['shipping_country'] == 'Singapore' else row['gross_sales'] / 0.62, axis=1)
df['net_sales_sgd'] = df.apply(lambda row: row['net_sales'] if row['shipping_country'] == 'Singapore' else row['net_sales'] / 0.62, axis=1)
df['total_sales_sgd'] = df.apply(lambda row: row['total_sales'] if row['shipping_country'] == 'Singapore' else row['total_sales'] / 0.62, axis=1)

# Index orders
index_order = ['orders','gross_sales_sgd', 'net_sales_sgd', 'total_sales_sgd', 'ordered_item_quantity', 'net_quantity', 'paid_ad_orders']
paid_ad_index_order = ['orders','gross_sales_sgd', 'net_sales_sgd', 'total_sales_sgd', 'ordered_item_quantity', 'net_quantity']

ValueError: Expected a 1D array, got an array with shape (0, 18)

## Paid ads

In [14]:
# paid ads
df_paid = df[(df['utm_campaign_medium'] == 'cpc') | (df['utm_campaign_medium'] == 'paid')]
paid_ad = pd.pivot_table(df_paid, index=['month'], values=['orders','gross_sales_sgd', 'net_sales_sgd', 'total_sales_sgd', 'ordered_item_quantity', 'net_quantity'], aggfunc='sum')
paid_ad.loc[Thismonth, 'paid_spends_sgd'] = paidspent_thismonth
paid_ad['Paid_CAC_sgd'] = paid_ad['paid_spends_sgd'] / paid_ad['orders']
# Transpose
paid_ad = paid_ad.transpose()

# change the index order
paid_ad = paid_ad.reindex(paid_ad_index_order)
# change the index 
paid_ad = paid_ad.reset_index()
paid_ad = paid_ad.rename(columns={'index': 'paid_ads'})
print(paid_ad)

KeyError: 'gross_sales_sgd'

## SG market

In [ ]:
# SG 
df_sg = df[df['shipping_country'] == 'Singapore']
# sg performance, breakdown by this month and next month
sg = pd.pivot_table(df_sg, index=['month'], values=['orders','gross_sales_sgd', 'net_sales_sgd', 'total_sales_sgd', 'ordered_item_quantity', 'net_quantity'], aggfunc='sum')
# Transpose
sg = sg.transpose()

# sg paid ads
df_paid_sg = df_sg[(df_sg['utm_campaign_medium'] == 'cpc') | (df_sg['utm_campaign_medium'] == 'paid')]
paid_ad_sg = pd.pivot_table(df_paid_sg, index=['month'], values=['orders'], aggfunc='sum')
# Transpose
paid_ad_sg = paid_ad_sg.transpose()
# change index name
paid_ad_sg = paid_ad_sg.rename(index={'orders': 'paid_ad_orders'})

# Combine together
df2 = pd.concat([sg,paid_ad_sg.loc[['paid_ad_orders']]],axis=0)

# change the index order
df2 = df2.reindex(index_order)
# change the index
df2 = df2.reset_index()
# change the column name
df2 = df2.rename(columns={'index': 'SG'})
df2

## UK market

In [4]:
# uk 
df_uk = df[df['shipping_country'] == 'United Kingdom']
# uk performance, breakdown by this month and next month
uk = pd.pivot_table(df_uk, index=['month'], values=['orders','gross_sales_sgd', 'net_sales_sgd', 'total_sales_sgd', 'ordered_item_quantity', 'net_quantity'], aggfunc='sum')
# Transpose
uk = uk.transpose()

# uk paid ads
df_paid_uk = df_uk[(df_uk['utm_campaign_medium'] == 'cpc') | (df_uk['utm_campaign_medium'] == 'paid')]
paid_ad_uk = pd.pivot_table(df_paid_uk, index=['month'], values=['orders'], aggfunc='sum')
# Transpose
paid_ad_uk = paid_ad_uk.transpose()
# change index name
paid_ad_uk = paid_ad_uk.rename(index={'orders': 'paid_ad_orders'})

# Combine together
df3 = pd.concat([uk,paid_ad_uk.loc[['paid_ad_orders']]],axis=0)

# change the index order
df3 = df3.reindex(index_order)
# change the index
df3 = df3.reset_index()
# change the column name
df3 = df3.rename(columns={'index': 'UK'})
df3

KeyError: 'gross_sales_sgd'

## Export the data

In [7]:
dfs = [df2, df3,paid_ad]
writer = pd.ExcelWriter('weekly_all-hands.xlsx', engine='xlsxwriter')
sheet_name = 'Sheet1'
start_row = 0
for a in dfs:
    a.to_excel(writer, sheet_name=sheet_name, index=False, startrow=start_row)
    start_row += a.shape[0] + 2
writer.save()